In [1]:
%load_ext autoreload
%autoreload 2
import os, sys
import glob

sys.path.append('/dartfs/rc/lab/F/FinnLab/tommy/isc_asynchrony_behavior/code/utils/')
sys.path.append('/dartfs/rc/lab/F/FinnLab/tommy/isc_asynchrony_behavior/code/modeling/joint-clm-prosody/')

from config import *
from src.data.prominence_regression_datamodule import ProminenceRegressionDataModule
from src.models.joint_clm_prosody import ProsodyCausalLM

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /dartfs/rc/lab/F/FinnLab/tommy/models/token
Login successful


/dartfs/rc/lab/F/FinnLab/tommy/conda/envs/prosody/lib/python3.12/site-packages/transformers/utils/hub.py:127: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


## Create PyTorch Lightning DataModule --> similar to pt dataloader

In [2]:
modeling_dir = os.path.join(BASE_DIR, 'code/modeling/joint-clm-prosody')

data_module = ProminenceRegressionDataModule(
    dataset_name = 'helsinki_prominence',
    data_dir = os.path.join(modeling_dir, 'data/heglsinki-prosody/data'),
    train_file = 'train_360.txt',
    val_file = 'dev.txt',
    test_file = 'test.txt',
    model_name = "gpt2",
    batch_size=32,
)

data_module.setup(stage="fit")

# for batch in data_module.train_dataloader():
#     sys.exit(0)

Using GPT2 tokenizer


/dartfs/rc/lab/F/FinnLab/tommy/conda/envs/prosody/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Dataloader: padding with token id: 50256
Loading data from /dartfs/rc/lab/F/FinnLab/tommy/isc_asynchrony_behavior/code/modeling/joint-clm-prosody/data/helsinki-prosody/data


Preprocessing samples: 100%|██████████| 109791/109791 [01:19<00:00, 1374.29it/s]


Failed 7217/109791


Preprocessing samples: 100%|██████████| 12199/12199 [00:08<00:00, 1462.63it/s]


Failed 768/12199


### Get all batches

In [4]:
import torch
from matplotlib import pyplot as plt
import seaborn as sns

In [5]:
all_labels = []

for batch in data_module.train_dataloader():
    all_labels.append(batch['tokenized_labels'].flatten())
all_labels = torch.concatenate(all_labels)

## Model code

In [11]:
import torch
from torch import nn
from torch.distributions.gamma import Gamma
from torch.distributions.normal import Normal
from torch.nn import functional as F

In [39]:
model = ProsodyCausalLM(model_name="gpt2", pretrained=True)
# outputs = test.forward(batch)

Loading Huggingface model.
Loading pretrained model
Using joint loss


In [5]:
loss, outputs = model.step(batch)

In [15]:
dist = Normal(outputs['mu'], outputs['var'].squeeze(-1))

In [6]:
logits, labels, mask = model.get_shifted_labels(
    logits=outputs['logits'],
    labels=batch['input_ids'],
    mask=batch['attention_mask']
)

# logits = outputs['logits']
# labels=batch['input_ids']
# mask=batch['attention_mask']

In [7]:
preds = torch.argmax(logits, -1)
correct = (preds == labels) * mask.bool()

correct = torch.sum(correct)
total = torch.sum(mask)

In [14]:
dist = outputs['dist']

In [35]:
# prosody labels
labels = batch['tokenized_labels']
dist = outputs['dist']

loss_mask = batch['loss_mask']

mu = outputs['mu']
var = outputs['var'].squeeze(-1)

In [70]:
from torch.distributions.gamma import Gamma

shifted_labels = labels[..., 1:].contiguous().view(-1)
shifted_mu = mu[..., :-1].contiguous() #.view(-1)
shifted_var = var[..., :-1].contiguous()#. view(-1)
shifted_mask = loss_mask[..., 1:].contiguous().view(-1)

dist = Gamma(shifted_mu, shifted_var)

In [83]:
batch['tokenized_labels']

tensor([[ 1.9840e+00, -9.9900e+02,  1.1410e+00,  ..., -1.0000e+00,
         -1.0000e+00, -1.0000e+00],
        [ 1.8000e-02,  2.0110e+00, -9.9900e+02,  ..., -1.0000e+00,
         -1.0000e+00, -1.0000e+00],
        [ 2.1030e+00,  2.2330e+00,  7.1000e-01,  ..., -1.0000e+00,
         -1.0000e+00, -1.0000e+00],
        ...,
        [ 1.4910e+00,  1.4910e+00,  1.9500e-01,  ..., -1.0000e+00,
         -1.0000e+00, -1.0000e+00],
        [ 2.4880e+00, -9.9900e+02,  1.0660e+00,  ..., -1.0000e+00,
         -1.0000e+00, -1.0000e+00],
        [-9.9900e+02, -9.9900e+02,  9.9400e-01,  ..., -1.0000e+00,
         -1.0000e+00, -1.0000e+00]])

In [85]:
batch['input_text']

['Then , across the pale glimmering of sand , Henriot saw a figure moving .',
 'The captain , who had so long been a cause of so much discomfort , was gone where the wicked cease from troubling .',
 'This first book of his brought him into notice , and served as an introduction to Tycho and to Galileo .',
 "Here's what I found at the very site of that final shipwreck !",
 'They stopped and stared at me .',
 "'Oh , indeed ! ' said mrs Rogers graciously ; for she was the lodger , and her servant was in waiting , so she was more gracious than intimate , in right of her position .",
 "Well , what of it ? It's not my fault . And he began thinking about the next day .",
 'Let us seek David on the hillsides , tending his flocks with loving care .',
 'But he heard a rustling in the branches , and a golden apple fell into his hand .',
 'So , comrades , said Myles at last , what shall we do now ?',
 'To acquire languages , departed or living in spite of such obstinacies as he now knew them inher

In [82]:
batch['original_labels']

[[1.984,
  None,
  1.141,
  0.0,
  1.745,
  0.858,
  0.003,
  1.229,
  None,
  1.309,
  0.596,
  0.07,
  0.954,
  0.57,
  None],
 [0.018,
  2.011,
  None,
  0.061,
  0.587,
  1.907,
  0.867,
  0.47,
  0.067,
  1.827,
  0.037,
  1.096,
  0.189,
  1.041,
  None,
  0.545,
  1.721,
  0.12,
  0.034,
  1.686,
  1.96,
  0.898,
  0.183,
  None],
 [2.103,
  2.233,
  0.71,
  0.112,
  0.866,
  1.541,
  0.603,
  0.056,
  0.842,
  None,
  0.195,
  2.098,
  0.113,
  0.029,
  1.691,
  0.044,
  1.588,
  0.212,
  0.018,
  0.718,
  None],
 [0.424,
  0.033,
  0.29,
  2.272,
  0.028,
  0.0,
  1.688,
  0.823,
  0.086,
  0.33,
  1.032,
  0.605,
  None],
 [0.508, 2.466, 0.003, 1.42, 0.0, 0.301, None],
 [None,
  None,
  2.403,
  None,
  None,
  4.155,
  0.32,
  0.487,
  0.404,
  None,
  0.971,
  0.165,
  1.14,
  0.145,
  0.0,
  None,
  1.249,
  0.465,
  0.0,
  2.27,
  0.562,
  0.0,
  None,
  1.075,
  0.351,
  0.75,
  0.026,
  0.293,
  1.181,
  0.008,
  None,
  0.789,
  0.282,
  0.725,
  0.012,
  0.072,
  None

In [79]:
batch.keys()

dict_keys(['input_text', 'tokenized_text', 'original_labels', 'tokenized_labels', 'input_ids', 'loss_mask', 'attention_mask', 'word_to_tokens'])

In [56]:
shifted_labels = shifted_labels * shifted_mask + 1e-4  # add small constant for numerical stability

In [60]:
nll = -dist.log_prob(shifted_labels)

In [61]:
masked_nll = nll * shifted_mask

In [65]:
loss = masked_nll.sum() / shifted_mask.sum()

In [66]:
loss

tensor(1.1118, grad_fn=<DivBackward0>)

In [42]:
shifted_mu

tensor([0.6996, 0.0008, 0.4233,  ..., 0.4181, 0.4164, 0.4170],
       grad_fn=<ViewBackward0>)

In [23]:
labels = labels * loss_mask + 1e-4  # add small constant for numerical stability
nll = -dist.log_prob(labels)

# mask loss
masked_nll = nll * loss_mask
masked_nll_mean = masked_nll.sum() / loss_mask.sum()

In [34]:
masked_nll

tensor([[ 2.1053, -0.0000,  1.6620,  ..., -0.0000, -0.0000, -0.0000],
        [-0.8416,  2.1437, -0.0000,  ..., -0.0000, -0.0000, -0.0000],
        [ 2.2663,  3.1142,  1.2448,  ..., -0.0000, -0.0000, -0.0000],
        ...,
        [ 1.4299,  1.6123,  1.0655,  ..., -0.0000, -0.0000, -0.0000],
        [ 2.4956, -0.0000,  3.3014,  ..., -0.0000, -0.0000, -0.0000],
        [-0.0000, -0.0000,  1.9905,  ..., -0.0000, -0.0000, -0.0000]],
       grad_fn=<MulBackward0>)

In [65]:
batch['loss_mask'].shape

torch.Size([32, 43])

In [62]:
outputs['var'].view(-1).contiguous()

tensor([1.4254, 5.9616, 3.6534,  ..., 1.2453, 1.2425, 1.2350],
       grad_fn=<ViewBackward0>)

In [112]:
print(5e-5 == 0.00005)

True


In [105]:
outputs['prosody_loss']

tensor(7.1849, grad_fn=<DivBackward0>)

### Manually perform forward pass

In [83]:
# tokenized labels = prosody values
input_embeds = test.get_input_embeddings(
    input_ids = batch['input_ids'], 
    prosody_values = batch['tokenized_labels']
)

# get outputs from causal LM
outputs = test.model.transformer(
    inputs_embeds=input_embeds, 
    attention_mask=batch["attention_mask"], 
)

# get the logits for predicting each item in the sequence
logits = test.model.lm_head(outputs.last_hidden_state)

# get prosody predictions
preds = test.regressor(outputs.last_hidden_state)

### Get distribution over the predictions

In [84]:
# split last dimension into mu and var
mu, var = torch.chunk(preds, chunks=2, dim=-1)

# ensure positivity of var + add a small constant for numerical stability
var = F.softplus(var)
var = (var + eps).squeeze(-1)

# have to squeeze the last dimension due to chunking
if test.output_activation is not None:
    mu = test.output_activation(mu.squeeze(-1))

# Gamma distribution with concentration mu and rate var
mu = F.softplus(mu)
dist = Gamma(mu, var)
preds = dist.mean

## Test natural output

In [100]:
test.prosody_loss(batch, outputs)

tensor(1.0272, grad_fn=<DivBackward0>)

In [94]:
# prosody labels
labels = batch['tokenized_labels']
dist = outputs['dist']
loss_mask = batch["loss_mask"]  # ignore padded sequence in loss



# log likelihood of labels given the distribution
labels = labels * loss_mask + 1e-4  # add small constant for numerical stability
nll = -dist.log_prob(labels)

# mask loss
masked_nll = nll * loss_mask
masked_nll_mean = masked_nll.sum() / loss_mask.sum()


In [95]:
masked_nll_mean

tensor(4.5416, grad_fn=<DivBackward0>)

## Test PEFT

In [21]:
from peft import get_peft_config, get_peft_model, LoraConfig, TaskType

In [584]:
lora_config = LoraConfig(
    r=16,
    lora_alpha=16,
    lora_dropout=0.1,
    task_type="CAUSAL_LM"
)

In [28]:
lora_config = {
    'r': 16,
    'lora_alpha': 32,
    'lora_dropout': 0.05,
    'bias': "none",
    'task_type': "CAUSAL_LM",
    'base_model_name_or_path': 'gpt2',
    'modules_to_save': []
}

In [27]:
config = LoraConfig(**lora_config)
model = get_peft_model(test.model, config)

model.print_trainable_parameters()

trainable params: 39,187,200 || all params: 163,627,008 || trainable%: 23.9491


In [587]:
# # peft
config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    base_model_name_or_path=model_name,
    # modules_to_save=["wte", "lm_head"]
)

model = get_peft_model(test.model, config)

In [576]:
# model_name = "gpt2"

# # loading base model and resizing embedding layers
# model = AutoModelForCausalLM.from_pretrained(model_name)

# # gradient checkpointing enabling
# model.enable_input_require_grads()
# model.gradient_checkpointing_enable()


# # peft
# config = LoraConfig(
#     r=16,
#     lora_alpha=32,
#     lora_dropout=0.05,
#     bias="none",
#     task_type="CAUSAL_LM",
#     base_model_name_or_path=model_name,
#     modules_to_save=["wte"] 
# )

# model = get_peft_model(model, config)

# model.print_trainable_parameters()

/dartfs/rc/lab/F/FinnLab/tommy/conda/envs/prosody/lib/python3.10/site-packages/peft/tuners/lora/layer.py:1091: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


In [591]:
# Inspect the trainable parameters
for name, param in test.named_parameters():
    if param.requires_grad:
        print(f"Trainable layer: {name}")
    else:
        print(f"Non-trainable layer: {name}")

Non-trainable layer: model.transformer.wte.weight
Non-trainable layer: model.transformer.wpe.weight
Non-trainable layer: model.transformer.h.0.ln_1.weight
Non-trainable layer: model.transformer.h.0.ln_1.bias
Non-trainable layer: model.transformer.h.0.attn.c_attn.base_layer.weight
Non-trainable layer: model.transformer.h.0.attn.c_attn.base_layer.bias
Trainable layer: model.transformer.h.0.attn.c_attn.lora_A.default.weight
Trainable layer: model.transformer.h.0.attn.c_attn.lora_B.default.weight
Non-trainable layer: model.transformer.h.0.attn.c_proj.weight
Non-trainable layer: model.transformer.h.0.attn.c_proj.bias
Non-trainable layer: model.transformer.h.0.ln_2.weight
Non-trainable layer: model.transformer.h.0.ln_2.bias
Non-trainable layer: model.transformer.h.0.mlp.c_fc.weight
Non-trainable layer: model.transformer.h.0.mlp.c_fc.bias
Non-trainable layer: model.transformer.h.0.mlp.c_proj.weight
Non-trainable layer: model.transformer.h.0.mlp.c_proj.bias
Non-trainable layer: model.transform

In [421]:
for name, param in test.model.named_parameters():
    print (name)

transformer.wte.weight
transformer.wpe.weight
transformer.h.0.ln_1.weight
transformer.h.0.ln_1.bias
transformer.h.0.attn.c_attn.base_layer.weight
transformer.h.0.attn.c_attn.base_layer.bias
transformer.h.0.attn.c_attn.lora_A.default.weight
transformer.h.0.attn.c_attn.lora_B.default.weight
transformer.h.0.attn.c_proj.weight
transformer.h.0.attn.c_proj.bias
transformer.h.0.ln_2.weight
transformer.h.0.ln_2.bias
transformer.h.0.mlp.c_fc.weight
transformer.h.0.mlp.c_fc.bias
transformer.h.0.mlp.c_proj.weight
transformer.h.0.mlp.c_proj.bias
transformer.h.1.ln_1.weight
transformer.h.1.ln_1.bias
transformer.h.1.attn.c_attn.base_layer.weight
transformer.h.1.attn.c_attn.base_layer.bias
transformer.h.1.attn.c_attn.lora_A.default.weight
transformer.h.1.attn.c_attn.lora_B.default.weight
transformer.h.1.attn.c_proj.weight
transformer.h.1.attn.c_proj.bias
transformer.h.1.ln_2.weight
transformer.h.1.ln_2.bias
transformer.h.1.mlp.c_fc.weight
transformer.h.1.mlp.c_fc.bias
transformer.h.1.mlp.c_proj.weigh

In [408]:
for layer in test.model.layers:
    if layer.trainable:
        print (layer.name)

AttributeError: 'GPT2LMHeadModel' object has no attribute 'layers'

In [394]:
TaskType.FEATURE_EXTRACTION

<TaskType.FEATURE_EXTRACTION: 'FEATURE_EXTRACTION'>

In [181]:
inputs = ['This is a']
# inputs = [f'{ins} {tokenizer.mask_token}' for ins in inputs]

inputs = tokenizer(inputs, return_tensors="pt")
# outputs = model(**inputs, output_hidden_states=True)